## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
from sklearn.preprocessing import OneHotEncoder


TensorFlow 2.x selected.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7) 
    return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu')) #32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, batch_size=128, epochs=100)

Train on 50000 samples
Epoch 1/100
50000/50000 [==============================] - 7s 144us/sample - loss: 1.2996 - accuracy: 0.5415
Epoch 2/100
50000/50000 [==============================] - 3s 54us/sample - loss: 0.9111 - accuracy: 0.6796
Epoch 3/100
50000/50000 [==============================] - 3s 53us/sample - loss: 0.7462 - accuracy: 0.7375
Epoch 4/100
50000/50000 [==============================] - 3s 54us/sample - loss: 0.6203 - accuracy: 0.7816
Epoch 5/100
50000/50000 [==============================] - 3s 54us/sample - loss: 0.5067 - accuracy: 0.8224
Epoch 6/100
50000/50000 [==============================] - 3s 55us/sample - loss: 0.4044 - accuracy: 0.8584
Epoch 7/100
50000/50000 [==============================] - 3s 54us/sample - loss: 0.3095 - accuracy: 0.8921
Epoch 8/100
50000/50000 [==============================] - 3s 52us/sample - loss: 0.2319 - accuracy: 0.9207
Epoch 9/100
50000/50000 [==============================] - 3s 54us/sample - loss: 0.1708 - accuracy: 0.9407
Epoc

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example = (np.zeros(shape=(1, 32, 32, 3)) - mean_train) / (std_train + 1e-7) 
classifier.predict(input_example)

array([[3.5483623e-17, 1.5384700e-33, 2.0838919e-12, 1.0000000e+00,
        1.8622630e-10, 5.2134496e-29, 9.0825598e-15, 8.5345494e-16,
        1.8913548e-14, 1.3397491e-26]], dtype=float32)